In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        np_XYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(np_XYs[:, 0]):
            XY = np_XYs[np_XYs[:, 0] == j][:, 1:]
            XYs.append(XY)
        path_XYs.append(XYs)
    return path_XYs

def plot(paths_XYs):
    fig, ax = plt.subplots(tight_layout=True, figsize=(8, 8))
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
    for i, XYs in enumerate(paths_XYs):
        c = colors[i % len(colors)]
        for XY in XYs:
            ax.plot(XY[:, 0], XY[:, 1], c=c, linewidth=2)
    ax.set_aspect('equal')
    plt.show()

def plot_i(XYs):
    fig, ax = plt.subplots(tight_layout=True, figsize=(8, 8))
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
    # for i, XYs in enumerate(paths_XYs):
    c = colors[2]
    for XY in XYs:
        ax.plot(XY[:, 0], XY[:, 1], c=c, linewidth=2)
    ax.set_aspect('equal')
    plt.show()

In [ ]:
from sklearn.decomposition import PCA

def pca_symmetry_axis(points):
    pca = PCA(n_components=2)
    pca.fit(points)
    # The first component gives us the direction of maximum variance
    axis_direction = pca.components_[0]
    # Get a point on the axis (mean of the points)
    center_point = pca.mean_
    return axis_direction, center_point

def project_points_onto_line(points, axis_direction, center_point):
    # Project points onto the axis
    projection = np.dot(points - center_point, axis_direction)
    return projection

# Example usage within the symmetry detection process
def find_pca_symmetry_axis(points):
    axis_direction, center_point = pca_symmetry_axis(points)
    return axis_direction, center_point


In [ ]:
def reflect_points_across_line(points, axis_direction, center_point):
    # Reflect the points across the line defined by axis_direction and center_point
    reflection = points - 2 * np.outer(np.dot(points - center_point, axis_direction), axis_direction)
    return reflection + center_point

def compute_symmetry_score(original_points, reflected_points):
    # Compute the average distance between the original and reflected points
    distances = np.linalg.norm(original_points - reflected_points, axis=1)
    return np.mean(distances)

def find_best_symmetry_axis(points):
    axis_direction, center_point = pca_symmetry_axis(points)
    
    # Initial reflection
    reflected_points = reflect_points_across_line(points, axis_direction, center_point)
    best_score = compute_symmetry_score(points, reflected_points)
    
    # Fine-tune the symmetry axis direction by small perturbations
    for angle in np.linspace(-np.pi/4, np.pi/4, 100):  # Rotate in small increments
        rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)],
                                    [np.sin(angle), np.cos(angle)]])
        new_axis_direction = np.dot(axis_direction, rotation_matrix)
        reflected_points = reflect_points_across_line(points, new_axis_direction, center_point)
        score = compute_symmetry_score(points, reflected_points)
        
        if score < best_score:
            best_score = score
            axis_direction = new_axis_direction
            
    return axis_direction, center_point, best_score


In [ ]:
from scipy.optimize import minimize

def symmetry_objective_function(params, points):
    # params contains the angle and offset of the symmetry line
    angle, offset_x, offset_y = params
    axis_direction = np.array([np.cos(angle), np.sin(angle)])
    center_point = np.array([offset_x, offset_y])
    
    reflected_points = reflect_points_across_line(points, axis_direction, center_point)
    score = compute_symmetry_score(points, reflected_points)
    return score

def optimize_symmetry_axis(points):
    # Initial guess: Use PCA result
    axis_direction, center_point = pca_symmetry_axis(points)
    initial_angle = np.arctan2(axis_direction[1], axis_direction[0])
    
    # Initial parameters: angle and center point coordinates
    initial_params = [initial_angle, center_point[0], center_point[1]]
    
    # Optimize the symmetry line
    result = minimize(symmetry_objective_function, initial_params, args=(points,),
                      method='BFGS', options={'disp': True})
    
    optimized_angle, optimized_x, optimized_y = result.x
    optimized_direction = np.array([np.cos(optimized_angle), np.sin(optimized_angle)])
    optimized_center_point = np.array([optimized_x, optimized_y])
    
    return optimized_direction, optimized_center_point


In [ ]:
import numpy as np

def remove_common_points(path_XYs, tolerance=1e-1):
    """
    Remove common points across all shapes in path_XYs.

    Parameters:
    - path_XYs: List of shapes, where each shape is a list of numpy arrays representing paths.
    - tolerance: The distance threshold below which points are considered the same.

    Returns:
    - List of shapes with common points removed.
    """
    
    def is_common_point(point, all_points):
        """Check if a point is common among all shapes."""
        for points in all_points:
            if np.any(np.all(np.isclose(points, point, atol=tolerance), axis=1)):
                continue
            else:
                return False
        return True
    
    # Collect all points from each shape
    all_points = [np.vstack(shape) for shape in path_XYs]

    # Create a new list to store shapes with common points removed
    new_path_XYs = []
    for shape in path_XYs:
        new_shape = []
        for path in shape:
            new_path = []
            for point in path:
                if not is_common_point(point, all_points):
                    new_path.append(point)
            if new_path:  # Only add non-empty paths
                new_shape.append(np.array(new_path))
        new_path_XYs.append(new_shape)
    
    return new_path_XYs


In [ ]:
def complete_shape_with_symmetry(points):
    # Find the best symmetry axis
    optimized_direction, optimized_center = optimize_symmetry_axis(points)
    
    # Reflect the points across this optimized axis
    reflected_points = reflect_points_across_line(points, optimized_direction, optimized_center)
    
    # Combine original and reflected points to complete the shape
    completed_points = np.vstack([points, reflected_points])
    return completed_points

# Example usage:
path_XYs  = read_csv(r'problems\occlusion1.csv') ##Input Goes here
# path_XYs = remove_common_points(path_XYs)
completed_paths = []
# XYs = path_XYs[1]
# completed_XYs=[]
# for XY in XYs:
#     completed_XYs.append(complete_shape_with_symmetry(XY))
#     completed_paths.append(completed_XYs)

for XYs in path_XYs:
    completed_XYs = []
    for XY in XYs:
        completed_XYs.append(complete_shape_with_symmetry(XY))
    completed_paths.append(completed_XYs)

plot(completed_paths)
# plot_i(path_XYs[4])
plot(path_XYs)


In [ ]:
import numpy as np

def write_csv(path_XYs, output_csv_path):
    csv_data = []
    polyline_id = 0
    
    for XYs in path_XYs:
        fragment_id = 0
        
        for XY in XYs:
            for point in XY:
                csv_data.append([polyline_id, fragment_id, point[0], point[1]])
            fragment_id += 1
        polyline_id += 1
    
    
    csv_data = np.array(csv_data)
    np.savetxt(output_csv_path, csv_data, delimiter=',', fmt='%d,%d,%f,%f')
path_XYs = completed_paths


#Output csv file is stored in output_csv_files folder. 
output_csv_path = r'output_csv_files\occlusion_sol.csv'
write_csv(path_XYs, output_csv_path)

In [ ]:
import svgwrite
import cairosvg
def polylines2svg ( paths_XYs , svg_path ):
    W , H = 0 , 0
    for path_XYs in paths_XYs :
        for XY in path_XYs :
            W , H = max (W , np .max ( XY [: , 0])) , max (H , np .max ( XY [: , 1]))
    padding = 0.1
    W , H = int ( W + padding * W ) , int ( H + padding * H )
# Create a new SVG drawing
    dwg = svgwrite . Drawing ( svg_path , profile = 'tiny' ,shape_rendering = 'crispEdges')
    group = dwg . g ()
    for i , path in enumerate ( paths_XYs ):
        path_data = []
        c = colours [ i % len( colours )]
        for XY in path :
            path_data . append (( " M " , ( XY [0 , 0] , XY [0 , 1])))
            for j in range (1 , len( XY )):
                path_data . append (( " L " , ( XY [j , 0] , XY [j , 1])))
            if not np . allclose ( XY [0] , XY [ -1]):
                path_data . append (( " Z " , None ))
        group . add ( dwg . path ( d = path_data , fill =c ,stroke = 'none'  , stroke_width =2))
    dwg . add ( group )
    dwg . save ()
    png_path = svg_path . replace ( '.svg' , '.png')
    fact = max (1 , 1024 // min (H , W ))
    cairosvg . svg2png ( url = r'output_images\occlusion.svg' , write_to = png_path ,parent_width =W , parent_height =H ,output_width = fact *W , output_height = fact *H ,background_color = 'white')
    return


polylines2svg(completed_paths)
